In [2]:
from parcels import FieldSet, ParticleSet, JITParticle, AdvectionRK4,ErrorCode, plotTrajectoriesFile,Variable,Geographic,GeographicPolar
from datetime import timedelta, datetime
import numpy as np
from operator import attrgetter
import math
from tqdm.autonotebook import tqdm
import xarray

/Users/jakecunningham/opt/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
file = xarray.open_dataset('/Volumes/4YP/Data/Eulerian_24hr/Eulerian20020101.nc')
print(file)

<xarray.Dataset>
Dimensions:    (latitude: 317, longitude: 720, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 2002-01-01
  * latitude   (latitude) float64 -78.0 -77.5 -77.0 -76.5 ... 79.0 79.5 80.0
  * longitude  (longitude) float64 -180.0 -179.5 -179.0 ... 178.5 179.0 179.5
Data variables:
    U          (time, latitude, longitude) float64 ...
    V          (time, latitude, longitude) float64 ...
Attributes:
    description:  Daily Mean Eulerian Current 20020101


In [4]:
file = xarray.open_dataset('/Volumes/JCHD/Microplastics_Paper/Data/Eulerian_24hr/Eulerian20020101.nc')
print(file)

<xarray.Dataset>
Dimensions:    (latitude: 317, longitude: 720, time: 1)
Coordinates:
  * time       (time) datetime64[ns] 2002-01-01
  * latitude   (latitude) float64 -78.0 -77.5 -77.0 -76.5 ... 79.0 79.5 80.0
  * longitude  (longitude) float64 -180.0 -179.5 -179.0 ... 178.5 179.0 179.5
Data variables:
    U          (time, latitude, longitude) float64 ...
    V          (time, latitude, longitude) float64 ...
Attributes:
    description:  Daily Mean Eulerian Current 20020101


In [5]:
#We can add or remove all the zeros according to preference. In case that they are left there, we only get daily data for the currents which will end up with the code running faster, but we do lose time resolution. Tests will determine if this loss in time resolution is actually important
filenames = {'U': "/Volumes/4YP/Data/Total_24hr_new/Total*.nc",
             'V': "/Volumes/4YP/Data/Total_24hr_new/Total*.nc",
             'uuss':"/Volumes/4YP/Data/Stokes_24hr/Stokes*.nc",
             'vuss':"/Volumes/4YP/Data/Stokes_24hr/Stokes*.nc",
             'Ue': "/Volumes/JCHD/Microplastics_Paper/Data/Eulerian_24hr/Eulerian*.nc",
             'Ve': "/Volumes/JCHD/Microplastics_Paper/Data/Eulerian_24hr/Eulerian*.nc",
             'borU':"/Volumes/4YP/Data/Boundary_Velocity_24hr/BV*.nc",
             'borV':"/Volumes/4YP/Data/Boundary_Velocity_24hr/BV*.nc"}
variables = {'U': 'eastward_eulerian_current_velocity',
             'V': 'northward_eulerian_current_velocity',
             'uuss':'uuss',
             'vuss':'vuss',
             'Ue': 'U',
             'Ve': 'V',
             'borU':'MaskUvel',
             'borV':'MaskVvel'}
dimensions = {'U':{'time':'time','lat':'latitude','lon':'longitude'},
              'V':{'time':'time','lat':'latitude','lon':'longitude'},
              'uuss':{'time':'time','lat':'latitude','lon':'longitude'},
              'vuss':{'time':'time','lat':'latitude','lon':'longitude'},
              'Ue': {'time':'time','lat':'latitude','lon':'longitude'},
              'Ve': {'time':'time','lat':'latitude','lon':'longitude'},
              'borU':{'time':'time','lat':'lat','lon':'lon'},
              'borV': {'time':'time','lat':'lat','lon':'lon'},
             }

In [6]:
#Create the fieldset with the periodic halo and time extrapolation for the EKE
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions,allow_time_extrapolation=True)
fieldset.add_periodic_halo(zonal=True)
fieldset.uuss.units=GeographicPolar()
fieldset.vuss.units=Geographic()
fieldset.Ue.units=GeographicPolar()
fieldset.Ve.units=Geographic()

In [7]:
#The starting coordinates of the Particles, for the North Pacific. They are generated
#by the code NAgrid.py, graciously send to me by David.
lons=np.load('/Volumes/4YP/Data/Particles_lon.npy')
lats=np.load('/Volumes/4YP/Data/Particles_lat.npy')
#lons, lats = np.meshgrid(lon,lat)
lons[lons>180]-=360

In [8]:
#And now we define what sort of particles we are actually dealing with
class SampleParticle(JITParticle):
#    #Now the part to determine the age of the particle
    Age=Variable('Age',initial=0.,dtype=np.float32)#agr is gonna be in seconds
    prev_time=Variable('prev_time',initial=attrgetter('time'),to_write=False)
    #Now the part to track the distance covered
#    distance = Variable('distance', initial=0., dtype=np.float32)
#    prev_lon = Variable('prev_lon', dtype=np.float32, to_write=False,
#                        initial=attrgetter('lon'))
#    prev_lat = Variable('prev_lat', dtype=np.float32, to_write=False,
#                        initial=attrgetter('lat'))
#    #Now I also want the particle to be deleted if it is on land (so it won't move)
#    count=Variable('count',initial=0,to_write=False)
#    init_lon = Variable('init_lon', dtype=np.float32, to_write=False,
#                        initial=attrgetter('lon'))
#    init_lat = Variable('init_lat', dtype=np.float32, to_write=False,
#                        initial=attrgetter('lat'))
#The starting point of the similation and the endtime

In [9]:
starttime=datetime(2002,1,1,0,0)
endtime=datetime(2014,12,31,21,0)
print(abs(endtime-starttime))
pset = ParticleSet(fieldset=fieldset, pclass=SampleParticle, lon=lons, lat=lats,time=starttime)

4747 days, 21:00:00


In [10]:
def DeleteParticle(particle, fieldset, time, dt):
    particle.delete()
    print('we deleted it at '+str(particle.lon)+' and '+str(particle.lat))

In [11]:
def AgeSample(particle, fiedset,time,dt):
    current_time=particle.time
    timedifference=current_time-particle.prev_time
    particle.Age+=timedifference
    particle.prev_time=current_time

In [12]:
def periodicBC(particle,fieldset,time,dt):
    if particle.lon<0:
        particle.lon+=360
    elif particle.lon >360:
        particle.lon-=360

In [13]:
def RungeKutta4FullCurrents(particle,fieldset,time,dt):
    lon0,lat0=particle.lon,particle.lat
    constant=0.00001*(-1)
    d=particle.depth
    u0=constant*fieldset.borU[time,lon0,lat0,d]+fieldset.U[time,lon0,lat0,d]+fieldset.uuss[time,lon0,lat0,d]+fieldset.Ue[time,lon0,lat0,d]
    v0=constant*fieldset.borV[time,lon0,lat0,d]+fieldset.V[time,lon0,lat0,d]+fieldset.vuss[time,lon0,lat0,d]+fieldset.Ve[time,lon0,lat0,d]

    lon1=lon0+u0*dt/2
    lat1=lat0+v0*dt/2
    u1=constant*fieldset.borU[time+0.5*dt,lon1,lat1,d]+fieldset.U[time+0.5*dt,lon1,lat1,d]+fieldset.uuss[time+0.5*dt,lon1,lat1,d]+fieldset.Ue[time+0.5*dt,lon1,lat1,d]
    v1=constant*fieldset.borV[time+0.5*dt,lon1,lat1,d]+fieldset.V[time+0.5*dt,lon1,lat1,d]+fieldset.vuss[time+0.5*dt,lon1,lat1,d]+fieldset.Ve[time+0.5*dt,lon1,lat1,d]

    lon2=lon0+u1*dt/2
    lat2=lat0+v1*dt/2
    u2=constant*fieldset.borU[time+0.5*dt,lon2,lat2,d]+fieldset.U[time+0.5*dt,lon2,lat2,d]+fieldset.uuss[time+0.5*dt,lon2,lat2,d]+fieldset.Ue[time+0.5*dt,lon2,lat2,d]
    v2=constant*fieldset.borV[time+0.5*dt,lon2,lat2,d]+fieldset.V[time+0.5*dt,lon2,lat2,d]+fieldset.vuss[time+0.5*dt,lon2,lat2,d]+fieldset.Ve[time+0.5*dt,lon2,lat2,d]
    
    lon3=lon0+u2*dt
    lat3=lat0+v2*dt
    u3=constant*fieldset.borU[time+dt,lon3,lat3,d]+fieldset.U[time+dt,lon3,lat3,d]+fieldset.uuss[time+dt,lon3,lat3,d]+fieldset.Ue[time+dt,lon3,lat3,d]
    v3=constant*fieldset.borV[time+dt,lon3,lat3,d]+fieldset.V[time+dt,lon3,lat3,d]+fieldset.vuss[time+dt,lon3,lat3,d]+fieldset.Ve[time+dt,lon3,lat3,d]

    particle.lon+=(u0+2*u1+2*u2+u3)/6. * dt
    particle.lat+=(v0+2*v1+2*v2+v3)/6. *dt

In [14]:
move=pset.Kernel(periodicBC)
Advection=pset.Kernel(RungeKutta4FullCurrents)
Agesam=pset.Kernel(AgeSample) 
totalKernal=Advection+move+Agesam

In [15]:
pfile = pset.ParticleFile(name="/Volumes/4YP/Data/Trajectories/Global_Total_Stokes_Eulerian",
                          outputdt=timedelta(hours=48))

Time=starttime
steps=0
while Time<=endtime:
    steps+=1
    Time+=timedelta(hours=48)

for i in tqdm(range(steps-1)):
    pset.execute(totalKernal,
                 runtime=timedelta(hours=48),  # runtime controls the interval of the plots
                 dt=timedelta(minutes=30),
                 recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle},
                 output_file=pfile
                 )  # the recovery kernel

INFO: Compiled SampleParticleRungeKutta4FullCurrentsperiodicBCAgeSample ==> /var/folders/kh/l0nftmqx1wsgmy0j289n8dgc0000gn/T/parcels-501/5fe55f358f2c83f7915be4474a0f86d8.so


In [16]:
file = xarray.open_dataset("/Volumes/4YP/Data/Trajectories/Global_Total_Stokes_Eulerian.nc")

In [17]:
print(file)

<xarray.Dataset>
Dimensions:     (obs: 2374, traj: 35145)
Dimensions without coordinates: obs, traj
Data variables:
    trajectory  (traj, obs) int32 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float32 ...
    lon         (traj, obs) float32 ...
    z           (traj, obs) float32 ...
    Age         (traj, obs) float32 ...
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        1.1.1
    parcels_mesh:           spherical


In [18]:
file.to_netcdf('/Volumes/4YP/Data/Trajectories/Global_Total_Stokes_Eulerian_NEW.nc')

In [18]:
h = xarray.open_dataset('/Volumes/4YP/Data/Trajectories/Global_Total_Stokes_NEW.nc')

In [19]:
print(h)

<xarray.Dataset>
Dimensions:     (obs: 2374, traj: 35145)
Dimensions without coordinates: obs, traj
Data variables:
    trajectory  (traj, obs) int32 ...
    time        (traj, obs) datetime64[ns] ...
    lat         (traj, obs) float32 ...
    lon         (traj, obs) float32 ...
    z           (traj, obs) float32 ...
    Age         (traj, obs) float32 ...
Attributes:
    feature_type:           trajectory
    Conventions:            CF-1.6/CF-1.7
    ncei_template_version:  NCEI_NetCDF_Trajectory_Template_v2.0
    parcels_version:        1.1.1
    parcels_mesh:           spherical
